# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Importing our CSV file from Part 1.
vacation_weather_df = pd.read_csv("/Users/azpunit/Desktop/python-api-challenge/WeatherPy/final_output_data/City_Data.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configuring gmaps.
gmaps.configure(api_key=g_key)

In [37]:
# Plotting our heatmap.
lat_lng_loc = vacation_weather_df[["Lat", "Lng"]]
humidity = vacation_weather_df["Humidity"]

layout = {"width": "900px",
          "height": "600px",
          "border": "1px solid black",
          "padding": "1px",
          "margin": "0 auto 0 auto"
          }

my_gmap = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))
heat_structure = gmaps.heatmap_layer(lat_lng_loc, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

my_gmap.add_layer(heat_structure)
my_gmap

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# Creating a new Dataframe filling weather criteria.
columns = ["City_ID", "City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]

intermediary_vacation_weather_df = vacation_weather_df.loc[(vacation_weather_df["Max Temp"] < 80) &
                                                           (vacation_weather_df["Max Temp"] > 70) &    
                                                           (vacation_weather_df["Wind Speed"] < 10) &                                        
                                                           (vacation_weather_df["Cloudiness"] == 0), columns]

final_vacation_weather_df = intermediary_vacation_weather_df.head(10)
final_vacation_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
91,91,Ponta do Sol,0,PT,1595626248,61,32.67,-17.10,71.01,3.94
128,128,Jales,0,BR,1595625960,32,-20.27,-50.55,74.89,5.84
177,177,Ribas do Rio Pardo,0,BR,1595626258,33,-20.44,-53.76,75.31,2.82
194,194,Beyneu,0,KZ,1595626260,42,45.32,55.20,76.86,7.76
214,214,Naviraí,0,BR,1595626110,43,-23.07,-54.19,76.77,5.55
224,224,Huilong,0,CN,1595626264,91,31.81,121.66,75.00,9.64
320,320,Mizdah,0,LY,1595626273,44,31.45,12.98,73.45,9.62
341,341,Tūkrah,0,LY,1595626276,84,32.53,20.58,75.96,5.23
350,350,Ust-Kuyga,0,RU,1595626276,48,70.02,135.60,76.86,2.48
357,357,Ilhabela,0,BR,1595626277,76,-23.78,-45.36,70.23,5.28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# store dataframe into the variable name hotel_df.
hotel_df = final_vacation_weather_df
hotel_df["Hotel Name"] = ""

In [29]:
import json

# search for hotels within 5000 meters.
params = {"radius": 5000,
          "types": "lodging",
          "keyword": "Hotel",
          "key": g_key
          }

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_address = requests.get(base_url, params=params)
    
    hotel_address = hotel_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map

figure_layout = {"width": "900px",
                 "height": "600px",
                 "border": "1px solid black",
                 "padding": "1px",
                 "margin": "0 auto 0 auto"
                 }

my_gmap = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

hotel_layer_mark = gmaps.marker_layer(locations, info_box_content=hotel_info)

my_gmap.add_layer(hotel_layer_mark)
my_gmap.add_layer(hotel_layer_mark)

# Display figure
my_gmap

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…